# VacationPy
----

In [10]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [11]:
# Path to the csv file
cities_csv = "../output_data/cities.csv"

# Ready the weather data for all cities into a DataFrame
cities_weather_df = pd.read_csv(cities_csv)

# Print DataFrame
cities_weather_df.head()

,City #,City,Country,Date,Latitude,Longitude,Max Temperature,Cloudiness,Humidity,Wind Speed
0,0,Esperance,AU,1580085724,-33.87,121.90,71.60,0,94,6.93
1,1,Acajutla,SV,1580085743,13.59,-89.83,81.21,13,74,10.09
2,2,Hobart,AU,1580086009,-42.88,147.33,63.00,75,48,10.29
3,3,Butaritari,KI,1580086070,3.07,172.79,82.56,100,79,11.83
4,4,Avarua,CK,1580086071,-21.21,-159.78,77.00,98,94,4.70


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Grab the latitude and longitude locations and humidity data 
locations = cities_weather_df[['Latitude', 'Longitude']]
humidity = cities_weather_df['Humidity'].astype(float)
max_humidity = humidity.max()

# Calculate the center point of all the geo-coordinates for the gmaps figure
center_lat = sum(cities_weather_df['Latitude']) / len(cities_weather_df['Latitude'])
center_long = sum(cities_weather_df['Longitude']) / len(cities_weather_df['Longitude'])
center_coordinates = (center_lat, center_long)

In [13]:
# Plot Heatmap
fig = gmaps.figure(center=center_coordinates, zoom_level=2)

# Create heat layer by using Lat and Lng as location and humidity as the weight
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=float(max_humidity),
                                 point_radius=1)

# Add heatmap layer to map
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
# Narrow down the DataFrame based on ideal weather conditions to return an ideal cities DataFrame
# A max temperature less than or equal to 80 degrees but higher than or equal to 65
# Wind speed less than or equal to 10 mph
# Not more than 30% cloud cover
# Not more than 80% humdity

ideal_weather_conditions_df = cities_weather_df[ (cities_weather_df['Max Temperature'] >= 65) & (cities_weather_df['Max Temperature'] <= 80) & (cities_weather_df['Wind Speed'] <= 10) & (cities_weather_df['Cloudiness'] <= 30)  & (cities_weather_df['Humidity'] <= 80)]

# Drop any rows with null values
ideal_cities_df = ideal_weather_conditions_df.dropna(how='any')
ideal_cities_df.count()

City #             18
City               18
Country            18
Date               18
Latitude           18
Longitude          18
Max Temperature    18
Cloudiness         18
Humidity           18
Wind Speed         18
dtype: int64

### Hotel Map

In [15]:
# Set Base URL for the Google Places API
places_api_url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

# Set up lists to hold hotel names, their geo-coordinates, city and country 
hotels_list = []
hotels_lat = []
hotels_lng = []
cities = []
countries = []

# Start of print log as we begin data retrieval using the API
print("Searching for Hotels...")
print(f"-" * 90)

# Iterate over the ideal cities DataFrame to find the first hotel for each city located within 5000 meters of the geo-coordinates.
for index, row in ideal_cities_df.iterrows(): 
    # Setup a dictionary for the parameters to do a 'nearbysearch' of hotels using the Google Places API
    # Parameters include:
    # - the lat/long of city around which to retrieve place information
    # - the radius (in meters) within which to return place results
    # - type of place, i.e. lodging/hotels that we want to search for
    params = { 'location': f"{row['Latitude']}, {row['Longitude']}", 'radius': 5000, 'type': 'lodging', 'key': g_key }
    
    # Execute the Google Place API call with the defined parameters and store the results into a JSON response object
    places_response = requests.get(places_api_url, params=params).json()

    # Try to grab the Hotel Name and it's Geo-Coordinates if available
    try:
        hotel_name = places_response['results'][0]['name']
        hotels_list.append(hotel_name)
        hotels_lat.append(places_response['results'][0]['geometry']['location']['lat'])
        hotels_lng.append(places_response['results'][0]['geometry']['location']['lng'])
        cities.append(row['City'])
        countries.append(row['Country'])
        # Print a log of each city as it's being processed with the city number and city name=
        print(f"Found the hotel '{hotel_name}' within 5000 meters of the city '{row['City']}'")

    # Handle exceptions for any hotels that could not be found within the 5000 meter radius 
    except:
        print(f"Cound not find any hotel within 5000 meters of the city '{row['City']}' | Skipping to the next one...")
        pass

# Print the final log indicating that the search has been completed
print("-" * 90)
print(f"Completed Hotel Search | Below is a list of all {len(hotels_list)} Hotels...")      
print("-" * 90)

# Save the Hotel results to a DataFrame and print it
hotel_df = pd.DataFrame( {'Hotel Name': hotels_list, 'Hotel Lat': hotels_lat, 'Hotel Lng': hotels_lng, 'City': cities, 'Country': countries} )
hotel_df

Searching for Hotels...
------------------------------------------------------------------------------------------
Found the hotel 'Observatory Guest House' within 5000 meters of the city 'Busselton'
Found the hotel 'CHACARA BAILLY' within 5000 meters of the city 'Castro'
Found the hotel 'Capy Motel' within 5000 meters of the city 'Suleja'
Found the hotel 'Holiday Inn Express & Suites Beeville' within 5000 meters of the city 'Beeville'
Cound not find any hotel within 5000 meters of the city 'San Ignacio Rio Muerto' | Skipping to the next one...
Found the hotel 'Debright Motel' within 5000 meters of the city 'Jalingo'
Found the hotel 'Norma franco' within 5000 meters of the city 'Tomatlan'
Found the hotel 'Whispering Bamboo Cove Resort' within 5000 meters of the city 'Morant Bay'
Found the hotel 'Austral Hotel' within 5000 meters of the city 'Comodoro Rivadavia'
Cound not find any hotel within 5000 meters of the city 'Qandala' | Skipping to the next one...
Found the hotel 'La Nena' with

,Hotel Name,Hotel Lat,Hotel Lng,City,Country
0,Observatory Guest House,-33.646393,115.348676,Busselton,AU
1,CHACARA BAILLY,-24.780711,-49.986828,Castro,BR
2,Capy Motel,9.200146,7.175102,Suleja,NG
3,Holiday Inn Express & Suites Beeville,28.412941,-97.725075,Beeville,US
4,Debright Motel,8.882003,11.373784,Jalingo,NG
5,Norma franco,19.934450,-105.251663,Tomatlan,MX
6,Whispering Bamboo Cove Resort,17.870329,-76.372712,Morant Bay,JM
7,Austral Hotel,-45.862680,-67.476643,Comodoro Rivadavia,AR
8,La Nena,22.521744,-79.469819,Caibarien,CU
9,Friends men's were,17.463867,77.419489,Chincholi,IN


In [16]:
# Using the template add the hotel markers to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# List of hotel marker locations represents the lat/long coordinates of each hotel
hotel_marker_locations = hotel_df[["Hotel Lat", "Hotel Lng"]]

In [17]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(hotel_marker_locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))